In [1]:
import sys
from sqlalchemy import create_engine
import pymysql
import pandas as pd

In [2]:
import random
random.seed(1030)

In [3]:
import numpy as np
import datetime
from datetime import timedelta, date

### Paths and Connection Details

In [6]:
exec(open("/Users/walkerag/Documents/osu/cs467/project_paths.py").read())

In [7]:
db_connection = create_engine(db_connection_str)

### Tables

In [21]:
all_tables = pd.read_sql('SHOW TABLES', con=db_connection)
print(all_tables)

# Describe all the DB tables
def describe_table(table, db_connection):
        
    print(table)    
    table_cols = pd.read_sql('DESCRIBE ' + table, con=db_connection)
    print(table_cols)

result = [describe_table(table, db_connection) for table in all_tables['Tables_in_project']]

           Tables_in_project
0                    Country
1                       Item
2             Price_Feedback
3             Price_Response
4              Shopping_List
5      Shopping_List_History
6                      State
7                      Store
8             Store_Feedback
9    Store_Feedback_Category
10            Store_Response
11                      User
12           User_Reputation
13  User_Reputation_Category
Country
          Field              Type Null  Key Default           Extra
0    country_id  int(10) unsigned   NO  PRI    None  auto_increment
1  country_name       varchar(50)   NO         None                
Item
              Field              Type Null  Key Default           Extra
0           item_id  int(10) unsigned   NO  PRI    None  auto_increment
1         item_name       varchar(50)   NO         None                
2          item_upc       varchar(12)   NO         None                
3  item_description      varchar(200)   NO         None     

In [31]:
query = """
    SELECT * from User
    WHERE user_id = 9
    LIMIT 10
    """

results_df = pd.read_sql(query, con=db_connection)
print(results_df)

   user_id first_name last_name                          email password  \
0        9    Melaine      Goff  melainegoff@notarealemail.com     None   

  signup_date last_login  user_country auth_token zip_code  
0  2020-04-20 2020-04-20           235       None     None  


In [27]:
query = """
    SELECT * from Store_Feedback
    ORDER BY time_added DESC
    LIMIT 10
    """

results_df = pd.read_sql(query, con=db_connection)
print(results_df)

   store_feedback_id  store_id  user_id          time_added  \
0               4596        67        9 2020-05-25 22:49:24   
1               4580        91       32 2020-04-24 00:00:00   
2               4584        72       75 2020-04-24 00:00:00   
3               4582        74       46 2020-04-24 00:00:00   
4               4578        74        1 2020-04-24 00:00:00   
5               4583        78       68 2020-04-24 00:00:00   
6               4581        74       46 2020-04-24 00:00:00   
7               4585        81       76 2020-04-24 00:00:00   
8               4579        81       23 2020-04-24 00:00:00   
9               4586        85       84 2020-04-24 00:00:00   

    store_feedback_text  store_feedback_category_id  
0  Prices were too high                           2  
1              Positive                          14  
2              Positive                           3  
3              Positive                           3  
4              Negative             

In [20]:
query = """
    SELECT * from Store
    LIMIT 10
    """

results_df = pd.read_sql(query, con=db_connection)
print(results_df)

   store_id    store_name  store_lat  store_long  store_active_flag  \
0        67  Trader Joe's  37.700004 -122.482237                  1   
1        68  Trader Joe's  37.664736 -122.446105                  1   
2        69  Trader Joe's  37.726647 -122.476312                  1   
3        70  Trader Joe's  37.805515 -122.413738                  1   
4        71  Trader Joe's  37.783370 -122.447874                  1   
5        72  Trader Joe's  37.790645 -122.417994                  1   
6        73       Safeway  37.768849 -122.427924                  1   
7        74       Safeway  37.776666 -122.394123                  1   
8        75       Safeway  37.782770 -122.431684                  1   
9        76       Safeway  37.796916 -122.398631                  1   

          store_street           store_city  store_state store_zip  
0  417 Westlake Center            Daly City            5     94015  
1   301 McLellan Drive  South San Francisco            5     94080  
2       265

In [18]:
query = """
    SELECT DISTINCT
    store_feedback_category
    , store_feedback_category_id
    FROM Store_Feedback_Category
    """

results_df = pd.read_sql(query, con=db_connection)

In [16]:
query = """
    (
    SELECT 
    item_name
    , 1 as item_found
    FROM Item
    WHERE item_upc = '1234'
    LIMIT 1
    )
    UNION ALL
    (
    SELECT 'Not Found' as item_name
    , 0 as item_found
    )
    LIMIT 1
    """

results_df = pd.read_sql(query, con=db_connection)

In [19]:
print(results_df)

   store_feedback_category  store_feedback_category_id
0            Out of Stocks                           1
1                  Pricing                           2
2           Rate Your Trip                           3
3                 Cashiers                           4
4                  Hygiene                           5
5                  Parking                           6
6                  Produce                           7
7                Selection                           8
8                 Checkout                           9
9                 Pharmacy                          10
10             Gas Station                          11
11                  Bakery                          12
12                    Deli                          13
13                 General                          14
14                 Returns                          15


### Load Data

In [44]:
# Function to load data from a CSV into MySQL database
def data_loader_csv(data_path, table_name, db_connection, load_type):
    
    print("Loading data from CSV")
    
    # Read in from CSV
    df = pd.read_csv(data_path + 'Input Data - ' + table_name.lower() + '.csv')
    
    print(df.head(5))
    
    # Load to mySQL
    df.to_sql(table_name, con = db_connection, if_exists = load_type, index = False)
    
    # Check table
    results_df = pd.read_sql('SELECT * FROM ' + table_name + ' LIMIT 5', con=db_connection)
    print(results_df)

In [ ]:
data_loader_csv(data_path, "Country", db_connection, "append")

In [ ]:
data_loader_csv(data_path, "State", db_connection, "append")

In [ ]:
data_loader_csv(data_path, "Item", db_connection, "append")

In [ ]:
data_loader_csv(data_path, "User", db_connection, "append")

In [ ]:
data_loader_csv(data_path, "Store", db_connection, "append")

In [ ]:
data_loader_csv(data_path, "Store_Feedback_Category", db_connection, "append")

In [343]:
data_loader_csv(data_path, "User_Reputation_Category", db_connection, "append")

Loading data from CSV
  user_reputation_category_name
0                    Apprentice
1                  Professional
2                        Master
3                         Elite
4                     Superstar
   user_reputation_category_id user_reputation_category_name
0                            1                    Apprentice
1                            2                  Professional
2                            3                        Master
3                            4                         Elite
4                            5                     Superstar


In [3]:
# cnx = mysql.connector.connect(user='root', password='cs467project',
#                               host='localhost',
#                               database='romanesco',
#                               use_pure=False)

In [ ]:
df = pd.read_sql('DELETE FROM Store', con=db_connection)
print(df)

In [48]:
df = pd.read_sql('DESCRIBE Country', con=db_connection)
print(df)

          Field              Type Null  Key Default           Extra
0    country_id  int(10) unsigned   NO  PRI    None  auto_increment
1  country_name       varchar(50)   NO         None                


In [ ]:
df = pd.read_sql('SELECT * FROM Country', con=db_connection)
print(df)

### Sklearn

In [11]:
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier

In [12]:
X, y = make_hastie_10_2(random_state=0)

In [15]:
X[1:10,1:10]

array([[ 1.45427351,  0.76103773,  0.12167502,  0.44386323,  0.33367433,
         1.49407907, -0.20515826,  0.3130677 , -0.85409574],
       [ 0.6536186 ,  0.8644362 , -0.74216502,  2.26975462, -1.45436567,
         0.04575852, -0.18718385,  1.53277921,  1.46935877],
       [ 0.37816252, -0.88778575, -1.98079647, -0.34791215,  0.15634897,
         1.23029068,  1.20237985, -0.38732682, -0.30230275],
       [-1.42001794, -1.70627019,  1.9507754 , -0.50965218, -0.4380743 ,
        -1.25279536,  0.77749036, -1.61389785, -0.21274028],
       [ 0.3869025 , -0.51080514, -1.18063218, -0.02818223,  0.42833187,
         0.06651722,  0.3024719 , -0.63432209, -0.36274117],
       [-0.35955316, -0.81314628, -1.7262826 ,  0.17742614, -0.40178094,
        -1.63019835,  0.46278226, -0.90729836,  0.0519454 ],
       [ 0.12898291,  1.13940068, -1.23482582,  0.40234164, -0.68481009,
        -0.87079715, -0.57884966, -0.31155253,  0.05616534],
       [ 0.90082649,  0.46566244, -1.53624369,  1.48825219,  1

In [16]:
X_train, X_test = X[:2000], X[2000:]

In [17]:
y_train, y_test = y[:2000], y[2000:]

In [18]:
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=1, random_state=0).fit(X_train, y_train)

In [19]:
clf.score(X_test, y_test)

0.913

### Data Simulation

In [220]:
# Function to create and upload a shopping list for a given user_id
# min_items and max_items limit number of items in list
def fill_shopping_list(user_id, min_items, max_items, items, add_date):
    
    # Determine total items in list
    tot_items = np.random.uniform(min_items, max_items, 1).round().astype(int)

    # Get item IDs
    list_items = np.random.choice(items['item_id'], tot_items, replace=False)

    # Convert to pandas dataframe
    dataset = pd.DataFrame({'item_id': list_items})

    # Format dataframe
    dataset['user_id'] = user_id
    dataset['time_added'] = add_date
    dataset['time_removed'] = None
    dataset = dataset[['user_id', 'item_id','time_added','time_removed']]

    # Load to mySQL
    # print("Loading")
    dataset.to_sql('Shopping_List_History', con = db_connection, if_exists = "append", index = False)

# Generate shopping list by sampling from items table
def generate_shopping_list_history(db_connection, min_items, max_items, add_date):
    
    # Get items
    items = pd.read_sql('SELECT DISTINCT item_id FROM Item', con=db_connection)
    
    # Users
    users = pd.read_sql('SELECT DISTINCT user_id FROM User', con=db_connection)
    
    # Loop over users
    result = [fill_shopping_list(user_id, min_items, max_items, items, add_date) for user_id in users['user_id']]

In [221]:
# ASSIGN STORE QUALITY
def assign_store_quality(chains, stores):
    
    print("Assigning store quality")

    # Assign overall chain quality
    # Higher is better
    chains['chain_quality'] = 0.5
    chains.loc[chains['store_name'] == "Trader Joe's", 'chain_quality'] = 0.7
    chains.loc[chains['store_name'] == "Safeway", 'chain_quality'] = 0.25
    chains.loc[chains['store_name'] == "Whole Foods Market", 'chain_quality'] = 0.8
    chains.loc[chains['store_name'] == "Costco", 'chain_quality'] = 0.55

    # Assign chain price level
    # Higher means more expensive
    chains['chain_price_level'] = 0.5
    chains.loc[chains['store_name'] == "Trader Joe's", 'chain_price_level'] = 0.6
    chains.loc[chains['store_name'] == "Safeway", 'chain_price_level'] = 0.3
    chains.loc[chains['store_name'] == "Whole Foods Market", 'chain_price_level'] = 0.8
    chains.loc[chains['store_name'] == "Costco", 'chain_price_level'] = 0.3

    # Assign out of stock level
    # Higher means more likely to have in-stock
    chains['chain_stock_level'] = 0.5
    chains.loc[chains['store_name'] == "Trader Joe's", 'chain_stock_level'] = 0.4
    chains.loc[chains['store_name'] == "Safeway", 'chain_stock_level'] = 0.6
    chains.loc[chains['store_name'] == "Whole Foods Market", 'chain_stock_level'] = 0.4
    chains.loc[chains['store_name'] == "Costco", 'chain_stock_level'] = 0.8
    
    print(chains.head())

    # Join on
    stores = stores.join(chains.set_index('store_name'), on='store_name')
    
    # Randomly move individual store attributes around overalll chain ones
    stores['store_quality'] = stores['chain_quality'] + np.random.normal(0, 0.05, stores.shape[0])
    stores['store_price_level'] = stores['chain_price_level'] + np.random.normal(0, 0.05, stores.shape[0])
    stores['store_stock_level'] = stores['chain_stock_level'] + np.random.normal(0, 0.05, stores.shape[0])
    
    print(stores.head())
    
    return(stores)

In [222]:
# ASSIGN ITEM PRICES
def assign_item_prices(items):
    
    print("Assigning item prices")
    items['item_price'] = np.random.poisson(5, items.shape[0]) + 1
    
    print(items.head())
    
    return(items)

In [223]:
# GENERATE PRICE FEEDBACK DATA
def generate_price_feedback(sim_date, sample_perc, user_shopping_list, stores, items, users, db_connection):

    # Join on preferred store
    user_shopping_list = user_shopping_list.join(users.set_index('user_id'), on='user_id')

    # Join on store attributes
    user_shopping_list = user_shopping_list.join(stores.set_index('store_id'), on='store_id')

    # Join on item price
    user_shopping_list = user_shopping_list.join(items.set_index('item_id'), on='item_id')

    # Limit to needed columns
    user_shopping_list = user_shopping_list[['user_id'
                                             , 'item_id'
                                             , 'store_id'
                                             , 'item_price'
                                             , 'store_price_level']]

    # Generate an item price based on default item price, store price level
    # Items can be +/- 20% of baseline
    user_shopping_list['item_price_level_perc'] = np.random.uniform(-0.2, 0.2, user_shopping_list.shape[0])

    # Assign sale flag
    user_shopping_list['sale_flag'] = 0
    user_shopping_list.loc[user_shopping_list['item_price_level_perc'] < -0.05, 'sale_flag'] = 1

    # Scale price by store price level (0.5 is average)
    # Forces a given store to be systematically higher or lower price...
    user_shopping_list['item_price_level_perc_adj'] = user_shopping_list['item_price_level_perc'] + ((user_shopping_list['store_price_level'] - 0.5) * 0.3)

    # Multiply default item price by price level to get final observed price
    # Round to 2 decimal places
    user_shopping_list['price'] = user_shopping_list['item_price'] * (1 + user_shopping_list['item_price_level_perc_adj'])
    user_shopping_list = user_shopping_list.round({'price': 2})
    
    # Sample a portion of the list
    feedback_sample = user_shopping_list.sample(frac=sample_perc, replace=False, random_state=1)

    # Arrange and format columns for table
    feedback_sample['price_currency'] = 'usd'
    feedback_sample['price_feedback_text'] = 'Dummy Price Data'
    feedback_sample['time_added'] = sim_date
    feedback_sample = feedback_sample[['store_id'
                                       , 'user_id'
                                       , 'item_id'
                                       , 'price'
                                       , 'price_currency'
                                       , 'sale_flag'
                                       , 'price_feedback_text'
                                       , 'time_added']]

    # Load feedback to MySQL
    feedback_sample.to_sql('Price_Feedback', con = db_connection, if_exists = "append", index = False)

In [224]:
def upload_store_feedback(user_store, store_feedback_category_id, db_connection):
    
    store_feedback = user_store[['store_id', 'user_id', 'time_added']].copy()

    # Feedback type
    store_feedback['store_feedback_category_id'] = store_feedback_category_id    
    
    # Determine if positive or negative
    if np.random.uniform(0, 1, 1) < float(user_store['quality_prob']):    
        print("Positive feedback!")
        store_feedback['store_feedback_text'] = 'Positive'
    else:
        print("Negative feedback!")
        store_feedback['store_feedback_text'] = 'Negative'    
            
    # Format for MySQL
    store_feedback = store_feedback[['store_id'
                                       , 'user_id'
                                       , 'time_added'
                                       , 'store_feedback_text'
                                       , 'store_feedback_category_id']]

    # Load feedback to MySQL
    store_feedback.to_sql('Store_Feedback', con = db_connection, if_exists = "append", index = False)  

# GENERATE STORE FEEDBACK DATA
def generate_store_feedback(sim_date, user_row, stores, db_connection):

    # Join on store attributes
    user_store = user_row.join(stores.set_index('store_id'), on='store_id')

    # Calculate probability of positive store feedback
    # Function of store quality and user preference
    user_store['quality_prob'] = user_store['store_quality'] * (user_store['user_preference'] / 0.5)

    # Time of feedback
    user_store['time_added'] = sim_date

    # Give general feedback
    if np.random.uniform(0, 1, 1) < float(user_store['user_activity']):

        print("Rating trip")
        upload_store_feedback(user_store, 14, db_connection)

    # Rate trip
    if np.random.uniform(0, 1, 1) < float(user_store['user_activity']):

        print("Rating trip")
        upload_store_feedback(user_store, 3, db_connection)

In [443]:
if np.random.uniform(0, 1, 1) < 0.05:
    print("Change preferred store")
    users.loc[users['user_id'] == u, 'store_id'] = np.random.choice(stores['store_id'], 1)

In [386]:
u

25

In [ ]:
users[users['user_id'] == u, 'store_id']

In [389]:
# GET USERS
users = pd.read_sql('SELECT user_id FROM User', con=db_connection)

# ASSIGN PREFERRED STORE
users = assign_preferred_store(users, stores)

Assigning a preferred store to users


In [445]:
print("User {u} taking a grocery trip!".format(u = u))

User 25 taking a grocery trip!


In [400]:
users.loc[users['user_id'] == u, 'store_id'] = 10

In [ ]:
users.iloc[users['user_id'] == u, 'store_id']

In [401]:
print("Change preferred store")
users.loc[users['user_id'] == u, 'store_id'] = np.random.choice(stores['store_id'], 1)

Change preferred store


In [318]:
def generate_store_response(sim_date, user_row, db_connection):
    
    # Get recent store feedback the user has not already responded to
    get_store_feedback_query = '''
        SELECT store_feedback_id
        FROM Store_Feedback 
        WHERE user_id != {user}
        AND store_id = {store}
        AND store_feedback_id
        NOT IN (SELECT store_feedback_id FROM Store_Response WHERE response_user_id = {user})
        AND time_added >= DATE_SUB(CAST('{date}' as DATE), INTERVAL 7 DAY)
        ORDER BY time_added DESC
        LIMIT 100
        '''
    
    get_store_feedback_query_fmt = get_store_feedback_query.format(
    user = user_row.iloc[0]['user_id']
    , store = user_row.iloc[0]['store_id']
    , date = sim_date)
    latest_store_feedback = pd.read_sql(get_store_feedback_query_fmt, con=db_connection)
    
    if latest_store_feedback.shape[0] > 0:

        print("Store feedback found")

        # Assign activity value
        latest_store_feedback['activity'] = np.random.uniform(0, 1, latest_store_feedback.shape[0])
        latest_store_feedback['preference'] = np.random.uniform(0, 1, latest_store_feedback.shape[0])

        # Assign upvote/downvote
        # More positive users more likely to assign an upvote
        latest_store_feedback['store_response_vote'] = 0
        latest_store_feedback.loc[latest_store_feedback['preference'] < float(user_row['user_preference']), 'store_response_vote'] = 1
        latest_store_feedback['store_response_text'] = 'Negative'
        latest_store_feedback.loc[latest_store_feedback['preference'] < float(user_row['user_preference']), 'store_response_text'] = 'Positive'

        # Filter based on how likely user is to respond
        latest_store_feedback_filtered = latest_store_feedback[latest_store_feedback["activity"] > float(user_row['user_activity'])].copy()

        # If response rows available, upload to MySQL
        if latest_store_feedback_filtered.shape[0] > 0:

            print("Giving store response!")
            latest_store_feedback_filtered['response_user_id'] = user_row.iloc[0]['user_id']
            latest_store_feedback_filtered['time_added'] = sim_date
            latest_store_feedback_filtered = latest_store_feedback_filtered[['store_feedback_id'
                                               , 'response_user_id'
                                               , 'time_added'
                                               , 'store_response_text'
                                               , 'store_response_vote']]

            # Load feedback to MySQL
            latest_store_feedback_filtered.to_sql('Store_Response', con = db_connection, if_exists = "append", index = False)

In [320]:
def generate_price_response(sim_date, user_row, db_connection):
    
    # Get recent price feedback the user has not already responded to
    get_price_feedback_query = '''
        SELECT price_feedback_id
        FROM Price_Feedback 
        WHERE user_id != {user}
        AND store_id = {store}
        AND price_feedback_id
        NOT IN (SELECT price_feedback_id FROM Price_Response WHERE response_user_id = {user})
        AND time_added >= DATE_SUB(CAST('{date}' as DATE), INTERVAL 7 DAY)
        ORDER BY time_added DESC
        LIMIT 100
        '''
    
    get_price_feedback_query_fmt = get_price_feedback_query.format(
    user = user_row.iloc[0]['user_id']
    , store = user_row.iloc[0]['store_id']
    , date = sim_date)
    latest_price_feedback = pd.read_sql(get_price_feedback_query_fmt, con=db_connection)
    
    if latest_price_feedback.shape[0] > 0:

        print("Price feedback found")

        # Assign activity value
        latest_price_feedback['activity'] = np.random.uniform(0, 1, latest_price_feedback.shape[0])
        latest_price_feedback['preference'] = np.random.uniform(0, 1, latest_price_feedback.shape[0])

        # Assign upvote/downvote
        # More positive users more likely to assign an upvote
        latest_price_feedback['price_response_vote'] = 0
        latest_price_feedback.loc[latest_price_feedback['preference'] < float(user_row['user_preference']), 'price_response_vote'] = 1
        latest_price_feedback['price_response_text'] = 'Negative'
        latest_price_feedback.loc[latest_price_feedback['preference'] < float(user_row['user_preference']), 'price_response_text'] = 'Positive'

        # Filter based on how likely user is to respond
        latest_price_feedback_filtered = latest_price_feedback[latest_price_feedback["activity"] > float(user_row['user_activity'])].copy()

        # If response rows available, upload to MySQL
        if latest_price_feedback_filtered.shape[0] > 0:

            print("Giving price response!")
            latest_price_feedback_filtered['response_user_id'] = user_row.iloc[0]['user_id']
            latest_price_feedback_filtered['time_added'] = sim_date
            latest_price_feedback_filtered = latest_price_feedback_filtered[['price_feedback_id'
                                               , 'response_user_id'
                                               , 'time_added'
                                               , 'price_response_text'
                                               , 'price_response_vote']]

            # Load feedback to MySQL
            latest_price_feedback_filtered.to_sql('Price_Response', con = db_connection, if_exists = "append", index = False)

In [225]:
def assign_preferred_store(users, stores):

    print("Assigning a preferred store to users")
    users['store_id'] = np.random.choice(stores['store_id'], users.shape[0], replace=True)
    
    return(users)

In [226]:
# Determine how likely a user is to assign positive or negative feedback
# Essentially assigning user biases
def assign_user_preferences(users):
    
    print("Assigning user preferences")    
    
    # How positive or negative a user is: closer to 1 means more likely to give positive feedback
    users['user_preference'] = np.random.uniform(0.1, 0.9, users.shape[0])
    
    # How active a user is: probability of adding or responding to feedback each trip
    users['user_activity'] = np.random.uniform(0.05, 0.6, users.shape[0])
    
    print(users.head())
    
    return(users)

In [227]:
# Update Shopping List table using Shopping List History
def update_shopping_list(db_connection):

    with db_connection.connect() as connection:
        result = connection.execute('''DELETE FROM Shopping_List''')
        
    with db_connection.connect() as connection:
        result = connection.execute('''
            INSERT INTO Shopping_List (user_id, item_id, time_added, shopping_list_history_id, item_quantity)
            SELECT
            user_id
            , item_id
            , time_added
            , shopping_list_history_id
            , 1 as item_quantity
            FROM Shopping_List_History
            WHERE time_removed IS NULL''')

In [228]:
# Seed shopper start date
def shopper_start_date(start_date, poisson_mean):
    
    return start_date + timedelta(int(np.random.poisson(poisson_mean, 1)))

In [293]:
def delete_table_rows(db_connection):
    
    print("Deleting table rows")

    # CLEAN OUT TABLES..
    with db_connection.connect() as connection:
        result = connection.execute('''DELETE FROM Price_Response''')

    with db_connection.connect() as connection:
        result = connection.execute('''DELETE FROM Price_Feedback''')

    with db_connection.connect() as connection:
        result = connection.execute('''DELETE FROM Store_Response''')    

    with db_connection.connect() as connection:
        result = connection.execute('''DELETE FROM Store_Feedback''')

    with db_connection.connect() as connection:
        result = connection.execute('''DELETE FROM Shopping_List''')

    with db_connection.connect() as connection:
        result = connection.execute('''DELETE FROM Shopping_List_History''')

    with db_connection.connect() as connection:
        result = connection.execute('''DELETE FROM User_Reputation''')        

In [356]:
# Determine a user's reputation category (rookie, elite, deity etc)
def assign_reputation_category(user_reputation):
    
    print("Assigning reputation category")
    
    user_reputation['user_reputation'] = user_reputation['user_received_upvotes'] + user_reputation['user_given_upvotes']

    user_reputation['user_reputation_category_id'] = 1
    user_reputation.loc[user_reputation['user_reputation'] > 5, 'user_reputation_category_id'] = 2
    user_reputation.loc[user_reputation['user_reputation'] > 10, 'user_reputation_category_id'] = 3
    user_reputation.loc[user_reputation['user_reputation'] > 20, 'user_reputation_category_id'] = 4
    user_reputation.loc[user_reputation['user_reputation'] > 50, 'user_reputation_category_id'] = 5
    user_reputation.loc[user_reputation['user_reputation'] > 100, 'user_reputation_category_id'] = 6
    
    return(user_reputation)

def generate_user_reputation(db_connection):
    
    print("Generating user reputation")
    
    # Responses given
    response_given = pd.read_sql('''
        SELECT
        response_user_id as user_id
        , SUM(CASE WHEN response_vote = 1 THEN 1 ELSE 0 END) as user_given_upvotes
        , SUM(CASE WHEN response_vote = 0 THEN 1 ELSE 0 END) as user_given_downvotes
        FROM 
        (
        SELECT response_user_id, store_response_vote as response_vote
        FROM Store_Response
        UNION ALL
        SELECT response_user_id, price_response_vote as response_vote    
        FROM Price_Response
        ) x
        GROUP BY response_user_id
      ''', con=db_connection)  

    # Responses received
    response_received = pd.read_sql('''
        SELECT
        user_id
        , SUM(CASE WHEN response_vote = 1 THEN 1 ELSE 0 END) as user_received_upvotes
        , SUM(CASE WHEN response_vote = 0 THEN 1 ELSE 0 END) as user_received_downvotes
        FROM 
        (
        SELECT sf.user_id, sr.store_response_vote as response_vote
        FROM Store_Response sr INNER JOIN Store_Feedback sf
        ON sr.store_feedback_id = sf.store_feedback_id
        UNION ALL
        SELECT pf.user_id, pr.price_response_vote as response_vote    
        FROM Price_Response pr INNER JOIN Price_Feedback pf
        ON pr.price_feedback_id = pf.price_feedback_id
        ) x
        GROUP BY user_id
      ''', con=db_connection)  

    # All users
    users = pd.read_sql('''
        SELECT user_id
        FROM User
        ''', con=db_connection)    
    
    # Join on
    user_reputation = users.join(response_given.set_index('user_id'), on='user_id', how = 'left')
    user_reputation = user_reputation.join(response_received.set_index('user_id'), on='user_id', how = 'left') 
    user_reputation = user_reputation.fillna(0)
    
    user_reputation['user_received_net'] = user_reputation['user_received_upvotes'] - user_reputation['user_received_downvotes']
    user_reputation['user_given_net'] = user_reputation['user_given_upvotes'] - user_reputation['user_given_downvotes']
    
    # Assign category
    user_reputation = assign_reputation_category(user_reputation)

    # Format and upload
    
    # Delete existing
    with db_connection.connect() as connection:
        result = connection.execute('''DELETE FROM User_Reputation''')
        
    # Arrange and format columns for table
    user_reputation = user_reputation[['user_id'
                                       , 'user_reputation'
                                       , 'user_reputation_category_id'
                                       , 'user_received_upvotes'
                                       , 'user_received_downvotes'
                                       , 'user_received_net'
                                       , 'user_given_upvotes'
                                       , 'user_given_downvotes'
                                       , 'user_given_net']]

    # Load feedback to MySQL
    print(user_reputation.head())
    user_reputation.to_sql('User_Reputation', con = db_connection, if_exists = "append", index = False)

In [ ]:
for n in reversed(range(day_count)):
    print(end_date - timedelta(n))

In [383]:
start_date

datetime.datetime(2020, 1, 26, 0, 0)

In [382]:
start_date = datetime.datetime(2020, 4, 25) - timedelta(90)

In [ ]:
# SIMULATE EACH DAY
end_date = datetime.datetime(2020, 4, 25)
for sim_date in (start_date - timedelta(n) for n in reversed(range(day_count))):
    print(sim_date)

In [358]:
# SIMULATION PARAMETERS
start_date = datetime.datetime(2020, 1, 1)
day_count = 60 # Length of simulation period
poisson_mean = 6
price_perc_items = 0.1 # Percentage of items to give price feedback on

# DELETE TABLE ROWS
delete_table_rows(db_connection)

# SEED USER SHOPPING LISTS
generate_shopping_list_history(db_connection, 1, 40, start_date)

# GET USERS
users = pd.read_sql('SELECT user_id FROM User', con=db_connection)

# GET STORES
stores = pd.read_sql('SELECT DISTINCT store_name, store_id FROM Store', con=db_connection)

# GET CHAINS
chains = pd.read_sql('SELECT DISTINCT store_name FROM Store', con=db_connection)

# GET ITEMS
items = pd.read_sql('SELECT item_id FROM Item', con=db_connection)

# GET STORE FEEDBACK
store_feedback = pd.read_sql('''SELECT store_feedback_category_id
                             , store_feedback_category FROM Store_Feedback_Category''', con=db_connection)

# ASSIGN STORE QUALITY
stores = assign_store_quality(chains, stores)

# ASSIGN ITEM PRICES
items = assign_item_prices(items)

# ASSIGN PREFERRED STORE
users = assign_preferred_store(users, stores)

# ASSIGN USER PREFERENCES
users = assign_user_preferences(users)

# SEED NEXT TRIP DATE
users['next_trip_date'] = df.apply(lambda row : shopper_start_date(start_date, poisson_mean), axis = 1)

# SIMULATE EACH DAY
for sim_date in (start_date + timedelta(n) for n in range(day_count)):
    
    print(sim_date)
    
    # Update shopping list data
    update_shopping_list(db_connection)
    
    # Update user reputation
    generate_user_reputation(db_connection)
    
    # Pull in current shopping lists
    all_shopping_list = pd.read_sql('SELECT * FROM Shopping_List', con=db_connection)
    
    # Loop over users
    for u in users['user_id']:
        
        # Get date of user's next grocery trip
        user_row = users[users['user_id'] == u]        
        next_trip_date = user_row.iloc[0]['next_trip_date']
        
        # If next trip is today, have the user take a grocery trip
        if next_trip_date == sim_date:
            
            print("Taking a grocery trip!")
            print(u)
            
            # Get user's shopping list
            user_shopping_list = all_shopping_list[all_shopping_list['user_id'] == u]
            
            # Generate store feedback
            generate_store_feedback(sim_date
                                    , user_row
                                    , stores
                                    , db_connection)
            
            # Generate price feedback
            generate_price_feedback(sim_date
                                    , price_perc_items
                                    , user_shopping_list
                                    , stores
                                    , items
                                    , users
                                    , db_connection)
            
            # Generate store response
            generate_store_response(sim_date
                                    , user_row
                                    , db_connection)
            
            # Generate price response
            generate_price_response(sim_date
                                    , user_row
                                    , db_connection)
            
            # Update date of next trip
            days_until_next_trip = int(np.random.poisson(poisson_mean, 1)) + 1
            users.loc[users['user_id'] == u, "next_trip_date"] = sim_date + timedelta(days_until_next_trip)

Deleting table rows
Assigning store quality
           store_name  chain_quality  chain_price_level  chain_stock_level
0        Trader Joe's           0.70                0.6                0.4
1             Safeway           0.25                0.3                0.6
2  Whole Foods Market           0.80                0.8                0.4
3              Costco           0.55                0.3                0.8
     store_name  store_id  chain_quality  chain_price_level  \
0  Trader Joe's        67            0.7                0.6   
1  Trader Joe's        68            0.7                0.6   
2  Trader Joe's        69            0.7                0.6   
3  Trader Joe's        70            0.7                0.6   
4  Trader Joe's        71            0.7                0.6   

   chain_stock_level  store_quality  store_price_level  store_stock_level  
0                0.4       0.686215           0.617951           0.386656  
1                0.4       0.634044           0.54

Price feedback found
Giving price response!
Taking a grocery trip!
108
Store feedback found
Giving store response!
Price feedback found
Giving price response!
Taking a grocery trip!
110
Taking a grocery trip!
117
Rating trip
Negative feedback!
Rating trip
Negative feedback!
Store feedback found
Giving store response!
Price feedback found
Giving price response!
Taking a grocery trip!
127
Rating trip
Negative feedback!
Rating trip
Positive feedback!
Taking a grocery trip!
131
Rating trip
Positive feedback!
Price feedback found
Taking a grocery trip!
133
Store feedback found
Giving store response!
Price feedback found
Giving price response!
Taking a grocery trip!
170
Rating trip
Negative feedback!
Taking a grocery trip!
171
2020-01-05 00:00:00
Generating user reputation
Assigning reputation category
   user_id  user_reputation  user_reputation_category_id  \
0        1              0.0                            1   
1        2              0.0                            1   
2        3  

Taking a grocery trip!
176
Rating trip
Positive feedback!
Price feedback found
Taking a grocery trip!
177
Taking a grocery trip!
179
Store feedback found
Giving store response!
Price feedback found
Giving price response!
Taking a grocery trip!
180
Taking a grocery trip!
181
Store feedback found
Giving store response!
Price feedback found
Giving price response!
2020-01-07 00:00:00
Generating user reputation
Assigning reputation category
   user_id  user_reputation  user_reputation_category_id  \
0        1              2.0                            1   
1        2              0.0                            1   
2        3              0.0                            1   
3        4              2.0                            1   
4        5              0.0                            1   

   user_received_upvotes  user_received_downvotes  user_received_net  \
0                    0.0                      0.0                0.0   
1                    0.0                      0.0      

ERROR:sqlalchemy.pool.impl.QueuePool:Exception during reset or similar
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/sqlalchemy/pool/base.py", line 693, in _finalize_fairy
    fairy._reset(pool)
  File "/usr/local/lib/python3.7/site-packages/sqlalchemy/pool/base.py", line 880, in _reset
    pool._dialect.do_rollback(self)
  File "/usr/local/lib/python3.7/site-packages/sqlalchemy/dialects/mysql/base.py", line 2337, in do_rollback
    dbapi_connection.rollback()
  File "/usr/local/lib/python3.7/site-packages/pymysql/connections.py", line 430, in rollback
    self._read_ok_packet()
  File "/usr/local/lib/python3.7/site-packages/pymysql/connections.py", line 394, in _read_ok_packet
    pkt = self._read_packet()
  File "/usr/local/lib/python3.7/site-packages/pymysql/connections.py", line 657, in _read_packet
    packet_header = self._read_bytes(4)
  File "/usr/local/lib/python3.7/site-packages/pymysql/connections.py", line 691, in _read_bytes
    data = s

KeyboardInterrupt: 

In [ ]:
def describe_table(table, db_connection):
        
    print(table)    
    table_cols = pd.read_sql('DESCRIBE ' + table, con=db_connection)
    print(table_cols)

result = [fill_list(user_id, min_items, max_items) for user_id in users['user_id']]

In [ ]:
pd.read_sql('''INSERT INTO Shopping_List (user_id, item_id, time_added, shopping_list_history_id, item_quantity)
                  SELECT 
                  user_id
                  , item_id
                  , time_added
                  , shopping_list_history_id
                  , 1 as item_quantity
                  FROM Shopping_List_History
                  WHERE time_removed IS NULL'''
                 , con=db_connection)

In [153]:
with db_connection.connect() as connection:
    result = connection.execute('''DELETE FROM Price_Feedback''')

In [109]:
with db_connection.connect() as connection:
    result = connection.execute('''DELETE FROM Shopping_List''')

In [ ]:
for row in result:
    print(row)

In [112]:
df = pd.read_sql('SELECT * FROM Shopping_List', con=db_connection)
print(df)

      shopping_list_id  user_id  item_id          time_added  \
0                 8192        1       52 2020-04-22 06:25:21   
1                 8193        1       10 2020-04-22 06:25:21   
2                 8194        1       27 2020-04-22 06:25:21   
3                 8195        1       42 2020-04-22 06:25:21   
4                 8196        1       33 2020-04-22 06:25:21   
5                 8197        1       15 2020-04-22 06:25:21   
6                 8198        1       31 2020-04-22 06:25:21   
7                 8199        1       17 2020-04-22 06:25:21   
8                 8200        1       43 2020-04-22 06:25:21   
9                 8201        1       22 2020-04-22 06:25:21   
10                8202        1        7 2020-04-22 06:25:21   
11                8203        1       14 2020-04-22 06:25:21   
12                8204        1       35 2020-04-22 06:25:21   
13                8205        1        5 2020-04-22 06:25:21   
14                8206        1        3

In [ ]:
df = pd.read_sql('DELETE FROM Shopping_List_History', con=db_connection)
print(df)

In [454]:
df = pd.read_sql('''SELECT store_name, item_id, AVG(price) as mean_price 
                 FROM Price_Feedback pf
                 INNER JOIN Store s on pf.store_id = s.store_id
                 GROUP BY 1, 2
                 ORDER BY item_id, store_name''', con=db_connection)
print(df)

             store_name  item_id  mean_price
0                Costco        1    4.682632
1               Safeway        1    4.835435
2          Trader Joe's        1    5.110750
3    Whole Foods Market        1    5.496842
4                Costco        2    5.625319
5               Safeway        2    5.639426
6          Trader Joe's        2    6.157273
7    Whole Foods Market        2    6.394839
8                Costco        3    5.414615
9               Safeway        3    5.519730
10         Trader Joe's        3    6.327692
11   Whole Foods Market        3    6.735610
12               Costco        4    7.080000
13              Safeway        4    7.731667
14         Trader Joe's        4    9.063043
15   Whole Foods Market        4    9.347143
16              Safeway        5    3.858571
17         Trader Joe's        5    4.087500
18   Whole Foods Market        5    4.219444
19               Costco        6    3.669259
20              Safeway        6    3.810303
21        

In [447]:
df = pd.read_sql('SELECT * FROM User_Reputation', con=db_connection)
print(df.describe())

       user_reputation_id     user_id  user_reputation  \
count          202.000000  202.000000       202.000000   
mean         25755.500000  101.500000       105.178218   
std             58.456537   58.456537        60.407224   
min          25655.000000    1.000000        10.000000   
25%          25705.250000   51.250000        63.000000   
50%          25755.500000  101.500000        93.000000   
75%          25805.750000  151.750000       133.500000   
max          25856.000000  202.000000       353.000000   

       user_reputation_category_id  user_received_upvotes  \
count                   202.000000             202.000000   
mean                      2.430693              52.589109   
std                       0.596548              39.074167   
min                       1.000000               0.000000   
25%                       2.000000              23.000000   
50%                       2.000000              42.500000   
75%                       3.000000              71

In [458]:
df = pd.read_sql('SELECT * FROM Store_Feedback LIMIT 100', con=db_connection)
print(df.head())

   store_feedback_id  store_id  user_id time_added store_feedback_text  \
0               2974        70       13 2020-01-27            Positive   
1               2975        70       13 2020-01-27            Negative   
2               2976        89      113 2020-01-27            Positive   
3               2977        80       15 2020-01-28            Negative   
4               2978        79       17 2020-01-28            Positive   

   store_feedback_category_id  
0                          14  
1                           3  
2                           3  
3                          14  
4                           3  


### Scrap

In [ ]:
# initialize list of lists 
data = [['United States of America'], ['Brazil'], ['Spain']] 
  
# Create the pandas DataFrame 
df = pd.DataFrame(data, columns = ['country_name']) 
  
# print dataframe. 
df 

In [4]:
import json

In [13]:
x =  '{"user_id": "121"}'

In [17]:
# parse x:
json_to_dict = json.loads(json.response)

# the result is a Python dictionary:
print(json_to_dict)

{'user_id': '121'}


In [18]:
json_to_dict['user_id']

'121'

In [40]:
# Get recent price feedback the user has not already responded to
get_top_stores_query = '''
    SELECT DISTINCT 
    s.store_id
    , s.store_name
    , s.store_street
    , s.store_city
    , s.store_zip
    , DATEDIFF(CURDATE(), sf.time_added) as test
    , CURDATE() - MAX(sf.time_added) as last_feedback
    FROM Store_Feedback sf
    INNER JOIN Store s
    ON sf.store_id = s.store_id
    WHERE sf.user_id = {user}
    GROUP BY 1, 2, 3, 4, 5
    ORDER BY sf.time_added DESC
    LIMIT 10
    '''
get_top_stores_query_fmt = get_top_stores_query.format(user = json_to_dict['user_id'])
print(get_top_stores_query_fmt)
user_favorite_stores = pd.read_sql(get_top_stores_query_fmt, con=db_connection)


    SELECT DISTINCT 
    s.store_id
    , s.store_name
    , s.store_street
    , s.store_city
    , s.store_zip
    , DATEDIFF(CURDATE(), sf.time_added) as test
    , CURDATE() - MAX(sf.time_added) as last_feedback
    FROM Store_Feedback sf
    INNER JOIN Store s
    ON sf.store_id = s.store_id
    WHERE sf.user_id = 121
    GROUP BY 1, 2, 3, 4, 5
    ORDER BY sf.time_added DESC
    LIMIT 10
    


In [65]:
print(user_favorite_stores)

   store_id store_name               store_street     store_city store_zip  \
0        97     Costco           4801 Central Ave       Richmond     94804   
1        82    Safeway  5290 Diamond Heights Blvd  San Francisco     94131   

   test   last_feedback store_name_fmt  
0    16 -20200403799490         costco  
1    93 -20200295799490        safeway  


In [64]:
user_favorite_stores['store_name_fmt'] = user_favorite_stores['store_name'].str.replace('[^a-zA-Z]', '').str.lower()

In [58]:
pd.read_sql(""" SHOW VARIABLES LIKE "%%version%%" """, con=db_connection)

,Variable_name,Value
0,innodb_version,5.7.22
1,protocol_version,10
2,slave_type_conversions,
3,tls_version,"TLSv1,TLSv1.1,TLSv1.2"
4,version,5.7.22-log
5,version_comment,Source distribution
6,version_compile_machine,x86_64
7,version_compile_os,Linux


In [45]:

user_favorite_stores = pd.read_sql("""
SELECT
s.store_id
, s.store_name
, REGEXP_REPLACE(s.store_name, '[^a-zA-Z0-9]+', '') as store_name_fmt
, s.store_street
, s.store_city
, s.store_zip
, SUM(1) as total_feedback
, COUNT(DISTINCT sf.user_id) as shoppers
FROM Store_Feedback sf
INNER JOIN Store s
ON sf.store_id = s.store_id
WHERE DATEDIFF(CURDATE(), sf.time_added) <= 30
GROUP BY 1, 2, 3, 4, 5, 6
ORDER BY total_feedback DESC
LIMIT 10
""", con=db_connection)


InternalError: (pymysql.err.InternalError) (1305, 'FUNCTION project.REGEXP_REPLACE does not exist')
[SQL: 
SELECT
s.store_id
, s.store_name
, REGEXP_REPLACE(s.store_name, '[^a-zA-Z0-9]+', '') as store_name_fmt
, s.store_street
, s.store_city
, s.store_zip
, SUM(1) as total_feedback
, COUNT(DISTINCT sf.user_id) as shoppers
FROM Store_Feedback sf
INNER JOIN Store s
ON sf.store_id = s.store_id
WHERE DATEDIFF(CURDATE(), sf.time_added) <= 30
GROUP BY 1, 2, 3, 4, 5, 6
ORDER BY total_feedback DESC
LIMIT 10
]
(Background on this error at: http://sqlalche.me/e/2j85)